In [1]:
import tensorflow as tf
print(tf.__version__)

ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (c:\Users\IT\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\eager\def_function.py)